<a href="https://colab.research.google.com/github/BalavSha/BalaArt/blob/main/BalaArt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center style="color:green;font-weight: bold;">BalaArt</center>

## **1. Data Collection and Preprocessing**

### **1.1: Downloading COCO Dataset**
Firstly, we need to download the COCO dataset from Kaggle. You can download it from this link: <a href="https://www.kaggle.com/c/coco-2017/data">Click Here</a>. <br>
Once you have downloaded the dataset, you need to unzip it and place it in a folder called BalaArt/COCO-Datasets/. <br><br>

Have you downloaded the dataset and placed it in the correct folder structure?

In [7]:
# trying
import json

# Set path to annotations file
annotations_file = "/content/drive/MyDrive/COCO_Datasets/coco2017/annotations/instances_train2017.json"

# Load annotations file
with open(annotations_file, 'r') as f:
    annotations = json.load(f)

# Extract captions for first 10 images
captions = []
for i, annotation in enumerate(annotations['annotations']):
    if i == 10:
        break
    captions.append(annotation['instance'])

# Save captions as new JSON file
with open('/content/drive/MyDrive/COCO_Datasets/coco2017/anno/instances_train2017.json', 'w') as f:
    json.dump(captions, f)

KeyError: ignored

*Downloading the Kaggle Dataset to the Google Drive and unzipping it:*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# run this only if Dataset in not already in Google Drive
"""
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = 'balavsharma'
os.environ['KAGGLE_KEY'] = '53676908461811eb1c957bde57bccab3'
!kaggle datasets download -d awsaf49/coco-2017-dataset -p "/content/drive/MyDrive/COCO_Datasets"
!unzip "/content/drive/MyDrive/COCO_Datasets/coco-2017-dataset.zip" -d "/content/drive/MyDrive/COCO_Datasets"

"""

'\n!pip install kaggle\nimport os\nos.environ[\'KAGGLE_USERNAME\'] = \'balavsharma\'\nos.environ[\'KAGGLE_KEY\'] = \'53676908461811eb1c957bde57bccab3\'\n!kaggle datasets download -d awsaf49/coco-2017-dataset -p "/content/drive/MyDrive/COCO_Datasets"\n!unzip "/content/drive/MyDrive/COCO_Datasets/coco-2017-dataset.zip" -d "/content/drive/MyDrive/COCO_Datasets"\n\n'

### **1.2 Preparing the Dataset for Training**

In [2]:
# import required libraries
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL

*Define the function to load and Preprocess the Images:*

In [6]:
def load_image(image_path):
  img = tf.io.read_file(image_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, (224, 224))
  img = tf.keras.applications.inception_v3.preprocess_input(img)
  return img

def load_dataset(data_dir, split):
  """
  data_dir: The directory containing the COCO dataset
  split: The split to use (train, test, validation)
  """
  data = tfds.load("coco/2017", split=split, data_dir=data_dir)
  images = []
  captions = []
  for example in data:
    img = load_image(example['image'])
    images.append(img)
    captions.append(example['caption'])
  return images, captions

*Now, load the training, testing & validation sets:*

In [7]:
data_dir = "/content/drive/MyDrive/COCO_Datasets/coco_datasets"

train_images, train_captions = load_dataset(data_dir, "batch_train")
test_images, test_captions = load_dataset(data_dir, "batch_test")
val_images, val_captions = load_dataset(data_dir, "batch_val")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

KeyboardInterrupt: ignored

### **1.3 Tokenizing the Captions**

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000, oov_token="<unk>", filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)
test_seqs = tokenizer.texts_to_sequences(test_captions)
val_seqs = tokenizer.texts_to_sequences(val_captions)

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

vocab_size = len(tokenizer.word_index)

NameError: ignored

*Pad the Sequences so that they all have same length by padding the sequences with zeros to a maximum length equal to the length of the longest sequence in the training set*